In [ ]:
# Using file 1
df = pd.read_csv('combined_data_1.txt', header = None, names = ['Cust_Id', 'Rating', 'Date'], usecols = [0,1,2])

In [ ]:
### appending new column movie_id to current dataset
df_nan = pd.DataFrame(pd.isnull(df.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

movie_np,movie_id = [],1


for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    temp = np.full((1,i-j-1), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)



In [ ]:
df= df[pd.notnull(df['Rating'])]

df['Movie_Id'] = movie_np.astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)


In [ ]:
df.head()  ### final data set with movie_id column

,Cust_Id,Rating,Date,Movie_Id
1,1488844,3.0,2005-09-06,1
2,822109,5.0,2005-05-13,1
3,885013,4.0,2005-10-19,1
4,30878,4.0,2005-12-26,1
5,823519,3.0,2004-05-03,1


In [ ]:
## sort on base of date
sorted_date = df.sort_values('Date')


In [ ]:
sorted_date.head()

,Cust_Id,Rating,Date,Movie_Id
9057969,510180,5.0,1999-11-11,1798
14895543,510180,3.0,1999-11-11,2866
19589582,510180,4.0,1999-11-11,3730
20397788,510180,2.0,1999-11-11,3870
6902840,510180,5.0,1999-11-11,1367


In [ ]:
length = sorted_date.shape[0]

train_size = int(length*0.93)  ### bcz of RAM issue
test_size = int(length*0.07)

In [ ]:
print(test_size, length)

1683763 24053764


In [ ]:
train_data = sorted_date.iloc[:test_size]  ### change it to train size
test_data = sorted_date.iloc[train_size+1300000:]
print(train_data.shape)
print(test_data.shape)


(1683763, 4)
(383764, 4)


In [ ]:
### drop date in both data
del train_data['Date']
del test_data['Date']

In [ ]:
test_data = test_data[["Cust_Id", "Movie_Id", "Rating"]]
train_data = train_data[["Cust_Id", "Movie_Id", "Rating"]]


In [ ]:
from scipy.linalg import sqrtm
def create_utility_matrix(data, formatizer = {'user':0, 'item': 1, 'value': 2}):
    """
        :param data:      Array-like, 2D, nx3
        :param formatizer:pass the formatizer
        :return:          utility matrix (n x m), n=users, m=items
    """
        
    itemField = formatizer['item']
    userField = formatizer['user']
    valueField = formatizer['value']
    userList = data.iloc[:,userField].tolist()
    itemList = data.iloc[:,itemField].tolist()
    valueList = data.iloc[:,valueField].tolist()
    users = list(set(data.iloc[:,userField]))
    items = list(set(data.iloc[:,itemField]))
    users_index = {users[i]: i for i in range(len(users))}
    pd_dict = {item: [np.nan for i in range(len(users))] for item in items}
    for i in range(0,len(data)):
        item = itemList[i]
        user = userList[i]
        value = valueList[i]
    pd_dict[item][users_index[user]] = value
    X = pd.DataFrame(pd_dict)
    X.index = users
        
    itemcols = list(X.columns)
    items_index = {itemcols[i]: i for i in range(len(itemcols))}
    # users_index gives us a mapping of user_id to index of user
    # items_index provides the same for items
    return X, users_index, items_index

In [ ]:
def svd(train, k):
    utilMat = np.array(train)
    # the nan or unavailable entries are masked
    mask = np.isnan(utilMat)
    masked_arr = np.ma.masked_array(utilMat, mask)
    item_means = np.mean(masked_arr, axis=0)
    # nan entries will replaced by the average rating for each item
    utilMat = masked_arr.filled(item_means)
    x = np.tile(item_means, (utilMat.shape[0],1))
    # we remove the per item average from all entries.
    # the above mentioned nan entries will be essentially zero now
    utilMat = utilMat - x
    # The magic happens here. U and V are user and item features
    U, s, V=np.linalg.svd(utilMat, full_matrices=False)
    s=np.diag(s)
    # we take only the k most significant features
    s=s[0:k,0:k]
    U=U[:,0:k]
    V=V[0:k,:]
    s_root=sqrtm(s)
    Usk=np.dot(U,s_root)
    skV=np.dot(s_root,V)
    UsV = np.dot(Usk, skV)
    UsV = UsV + x
    print("svd done")
    return UsV

In [ ]:
def rmse(true, pred):
    # this will be used towards the end
    x = true - pred
    return sum([xi*xi for xi in x])/len(x)
# to test the performance over a different number of features
no_of_features = [8]#,10,12,14,17]
utilMat, users_index, items_index = create_utility_matrix(train_data)
error_sum = 0
leng = 0
for f in no_of_features: 
    svdout = svd(utilMat, k=f)
    pred = [] #to store the predicted ratings
    for _,row in test_data.iterrows():
        user = row['Cust_Id']
        item = row['Movie_Id']
        try:
            pred_rating = 0
            u_index = users_index[int(user)]
            if item in items_index:
                i_index = items_index[item]
                pred_rating = svdout[u_index, i_index][0]
            else:
                pred_rating = np.mean(svdout[u_index, :])
            #pred.append(pred_rating)
            error_sum += (row["Rating"] - pred_rating)**2
            leng+=1
            #print(row["Rating"], pred_rating, (row["Rating"] - pred_rating)**2)
        except:
            pass
#print(rmse(test_data['Rating'], pred))

In [ ]:
import math
math.sqrt(error_sum/leng)